# Wandle Datensatz in das Format für YoloX und Yolov8 um

In [60]:
import os
import shutil
import json

import numpy as np
import pandas as pd

from pycocotools import coco

## YoloX

In [52]:
datasetPathSRC = '../data/dataFiltered'
datasetPathYoloX = '../data/dataFormats/dataYoloX'

# Erstelle Ordnerstruktur wenn nicht vorhanden
if not os.path.exists(datasetPathYoloX):
    os.makedirs(datasetPathYoloX)

In [53]:
# Erstelle leeren COCO-Datensatz
cocoDataset = coco.COCO()

In [54]:
# Erstelle COCO Dataset categories
categories = [
    {'id': 0, 'name': 'car'},
    {'id': 1, 'name': 'pedestrian'},
    {'id': 2, 'name': 'trafficLight'},
    {'id': 3, 'name': 'truck'},
    {'id': 4, 'name': 'biker'}
    ]

categoryIdMap = {category['name']: category['id'] for category in categories}

categoryData = [{'id': category['id'], 'name': category['name']} for category in categories]
cocoDataset.dataset['categories'] = categoryData

print(cocoDataset.dataset['categories'])


[{'id': 0, 'name': 'car'}, {'id': 1, 'name': 'pedestrian'}, {'id': 2, 'name': 'trafficLight'}, {'id': 3, 'name': 'truck'}, {'id': 4, 'name': 'biker'}]


In [58]:
# Füge Bilder zum COCO-Datensatz hinzu
annotationFilePath = os.path.join(datasetPathSRC, '_annotations_newClasses.csv')
imagePath = os.path.join(datasetPathSRC)

annotationFile = pd.read_csv(annotationFilePath)
cocoDataset.dataset['images'] = []
cocoDataset.dataset['annotations'] = []

for index, row in annotationFile.iterrows():
    # Pfad zum Bild
    imagePath = os.path.join(datasetPathSRC, row['filename'])

    # Kopiere Bild in YoloX-Ordner
    imagePathYoloX = os.path.join(datasetPathYoloX, row['filename'])
    if not os.path.exists(imagePathYoloX):
        shutil.copy(imagePath, imagePathYoloX)


    imageData = {
        'id': index,
        'file_name': row['filename'],
        'width': row['width'],
        'height': row['height']
    }
    cocoDataset.dataset['images'].append(imageData)

    # Füge Annotationen zum COCO-Datensatz hinzu
    annotationData = {
        'id': index,
        'image_id': index,
        'category_id': categoryIdMap[row['class']],
        'bbox': [row['xmin'], row['ymin'], row['xmax'] - row['xmin'], row['ymax'] - row['ymin']],
        'area': (row['xmax'] - row['xmin']) * (row['ymax'] - row['ymin']),
        'iscrowd': 0
    }
    cocoDataset.dataset['annotations'].append(annotationData)
    
cocoDataset.createIndex()

# Speichere COCO-Datensatz
cocoData = cocoDataset.dataset
with open(os.path.join(datasetPathYoloX, '_annotations.json'), 'w') as jsonFile:
    json.dump(cocoData, jsonFile)


creating index...
index created!


In [47]:
# Ausgabe von Beispielen aus dem COCO-Datensatz
print(cocoDataset.dataset['images'][0])
cocoDataset.dataset['annotations'][3]

{'id': 0, 'file_name': '1478900859981702684_jpg.rf.6830635c7d9197475638f0818f5dd103.jpg', 'width': 512, 'height': 512}


{'id': 3,
 'image_id': 3,
 'category_id': 0,
 'bbox': [25, 258, 81, 46],
 'area': 3726,
 'iscrowd': 0}

## Yolov8

In [ ]:
datasetPathSRC = '../data/dataFiltered'
datasetPathYolov8 = '../data/dataFormats/dataYoloV8'